In [ ]:
# import libraries
import numpy as np
import sys
import psi4
from helper_PFCI import PFHamiltonianGenerator
np.set_printoptions(threshold=sys.maxsize)
#psi4.core.set_output_file('output.dat', False)
import time
from scipy.optimize import curve_fit
from scipy import interpolate
from matplotlib import pyplot as plt
from numpy.polynomial import Chebyshev
from scipy import constants
from numpy.polynomial import Polynomial

In [ ]:
# setup basic arguments for qed-ci calculation

# z-matrix for H2
mol_str_H2 = """
H
H 1 0.74
symmetry c1
"""

# options for the PFHamiltonian Generator class - include cavity effects
cavity_dict = {
    'omega_value' : 0.12086,
    'lambda_vector' : np.array([0, 0, 0.05]),
    'ci_level' : 'fci',   
    'full_diagonalization' : True,
    'number_of_photons' : 1, #<== this is a minimal photon basis, should explore increasing this 
}

# options for PFHamiltonian Generator class - exclude cavity effects
cavity_free_dict = {
    'omega_value' : 0.0,
    'lambda_vector' : np.array([0, 0, 0.0]),
    'ci_level' : 'fci',   
    'full_diagonalization' : True,
    'number_of_photons' : 0, 
}



In [ ]:
mol_tmpl = """
H
H 1 **R**
symmetry c1
"""
options_dict = {
    "basis": "cc-pVQZ",
    "scf_type": "pk",
    "e_convergence": 1e-10,
    "d_convergence": 1e-10,
    'num_roots' : 2
}
r_data = np.linspace(0.6, 0.9, 50)
psi4.set_options(options_dict)
fci_S0 = []
fci_S1 = []
for r in r_data:
    mol_str = mol_tmpl.replace("**R**", str(r))
    mol = psi4.geometry(mol_str)
    scf_e, wfn = psi4.energy('SCF', return_wfn=True)
    fci_energy, wfn = psi4.energy('fci',ref_wfn=wfn, return_wfn=True)
    fci_S0.append(wfn.variable("CI ROOT 0 TOTAL ENERGY"))
    fci_S1.append(wfn.variable("CI ROOT 1 TOTAL ENERGY"))

In [ ]:
mol_tmpl_e = """
H
H 1 **R**
symmetry c1
"""

r_data_e = np.linspace(1.15, 1.45, 50)
psi4.set_options(options_dict)
fci_S0_e = []
fci_S1_e = []
for r in r_data_e:
    mol_str_e = mol_tmpl.replace("**R**", str(r))
    mol_e = psi4.geometry(mol_str_e)
    scf_e, wfn = psi4.energy('SCF', return_wfn=True)
    fci_energy, wfn = psi4.energy('fci',ref_wfn=wfn, return_wfn=True)
    fci_S0_e.append(wfn.variable("CI ROOT 0 TOTAL ENERGY"))
    fci_S1_e.append(wfn.variable("CI ROOT 1 TOTAL ENERGY"))

In [ ]:
# z-matrix for H2
mol_str_H2 = """
H
H 1 0.74
symmetry c1
"""

options_dict = {
    "basis": "cc-pVQZ",
    "scf_type": "pk",
    "e_convergence": 1e-10,
    "d_convergence": 1e-10,
    'num_roots' : 1
}

# let's run a psi4 geometry optimization to get the optimal geometry

# set the options
psi4.set_options(options_dict)

# first set the coordinates in init_string as the molecule's geometry 
mol0_H2 = psi4.geometry(mol_str_H2)

# now capture the psi4 geometry data with the guess coordinates
init_geometry_string_H2 = psi4.core.Molecule.create_psi4_string_from_molecule(mol0_H2)

# run the geometry optimization
psi4.optimize("fci")

guess_geometry_H2 = mol0_H2.geometry()
print(guess_geometry_H2)

guess_string_H2 = psi4.core.Molecule.create_psi4_string_from_molecule(mol0_H2)


Going to use the results of the geometry optimization that found $r_{eq} = 0.741783$ Angstroms and
esimate the force constant using the [5-point stencil approach](https://en.wikipedia.org/wiki/Five-point_stencil):

$$ f''(r_{eq}) \approx \frac{-f(r_{eq} + 2h) + 16f(r_{eq}+h) - 30f(r_{eq}) + 16f(r_{eq}-h) - f(r_{eq}-2h)}{12 h^2} $$
where $h$ is a small displacement.  

In [ ]:
# z-matrix for H2
mol_str_H2 = """
units au
H
H 1 **R**
symmetry c1
"""

# define equilibrium bondlength in Bohr
r_eq = 0.741783 * 1.8897259886

# define displacement
h_au = 0.0005 * 1.8897259886
r_disp_au = np.array([r_eq + 2*h_au, r_eq + h_au, r_eq, r_eq - h_au, r_eq - 2*h_au])
fci_disp_au = []

for r in r_disp_au:
    mol_str_e = mol_tmpl.replace("**R**", str(r))
    mol_e = psi4.geometry(mol_str_e)
    scf_e, wfn = psi4.energy('SCF', return_wfn=True)
    fci_energy, wfn = psi4.energy('fci',ref_wfn=wfn, return_wfn=True)
    fci_disp_au.append(wfn.variable("CI ROOT 0 TOTAL ENERGY"))
    

k = (-fci_disp[0] + 16 * fci_disp[1] - 30 * fci_disp[2] + 16 * fci_disp[3] - fci_disp[4]) / (12 * h ** 2)
k_au = (-fci_disp_au[0] + 16 * fci_disp_au[1] - 30 * fci_disp_au[2] + 16 * fci_disp_au[3] - fci_disp_au[4]) / (12 * h_au ** 2)


In [ ]:
harm = 1/2 * k * (r_disp - r_disp[2]) ** 2 + fci_disp[2]
harm_au = 1/2 * k_au * (r_disp_au - r_disp_au[2]) ** 2 + fci_disp_au[2]
#plt.plot(r_disp_au, fci_disp, 'ro')
plt.plot(r_disp_au, fci_disp_au, 'b*')
plt.plot(r_disp_au, harm_au, 'blue')
plt.show()
print(k)

In [ ]:
# set the options
#psi4.set_options(options_dict)

# first set the coordinates in guess_string as the molecule's geometry 
mol_H2 = psi4.geometry(mol_str_H2)

# now capture the psi4 geometry data with the guess coordinates
#guess_geometry_string_H2 = psi4.core.Molecule.create_psi4_string_from_molecule(mol_H2)

# run the geometry optimization
#psi4.optimize("fci")

#opt_geometry = mol_H2.geometry()

# capture the psi4 geometry data from the optimization
#opt_geometry_string_H2 = psi4.core.Molecule.create_psi4_string_from_molecule(mol_H2)

# run excited-state calculations - get first 5 excited states
#n_states = 5

# run psi4 TDSCF at same level of theory as geometry optimization
psi4_rhf_e, wfn_H2 = psi4.frequency("ccsd/cc-pVQZ", return_wfn=True, molecule=mol_H2,maxiter=50)



## Calculation of k
Fit ground state PES of H2 to a quintic polynomial

Calculated value of k is very close to literature value (~$575 N/m$), so I am now conficent that the issue is in $ x_0 $

In [ ]:
# Constants and Variables
au_to_SI = (4.35974 * 10 ** (-18)) * 10 ** 20
min_S0_loc = np.argmin(fci_S0[:])
min_S1_loc = np.argmin(fci_S1_e[:])
r_eq_ang = r_data[min_S0_loc]
print(f'Min on S0 is {r_data[min_S0_loc]}')
print(f'Min on S1 is {r_data_e[min_S1_loc]}')

# Fitting S0 PES to a quintic polynomial
poly = np.poly1d(np.polyfit(r_data, fci_S0, 5))
print(poly)

#Taking first and second derivative of S0 PES and evaluating at r_eq
first_derivative = poly.deriv()
second_derivative = first_derivative.deriv()
k_test_au = second_derivative(r_eq_ang)
k_test_SI = k_test_au * au_to_SI
print(k_test_SI)

#plotting S0 PES and quintic fit
plt.plot(r_data, poly(r_data), 'm-', label='fit')
plt.plot(r_data, fci_S0[:], 'bo', label='cavity free |g>')
plt.show

## Calculation of $ \text{x}_0 $

$$ \frac{\hbar}{2}\sqrt{\frac{k}{\mu}} = \frac{k}{2}(x_0 - x_{eq})^2 + V_0 $$

Expanded, solved for $ x_o $, and found zeros using quadratic formula

In [ ]:
mu_amu = 1.007 / 2
mu_kg = mu_amu  * 10 ** (-3) / (6.022 * 10 ** 23) 
r_eq_SI = r_eq_ang * 10 ** (-10)
h_bar = constants.hbar
V_0_loc = np.argmin(fci_S0)
V_0 = fci_S0[V_0_loc] * 4.35974 * 10 ** (-18)
left = (h_bar / 2) * np.sqrt(k_test_SI / mu_kg)
a = 0.5 * k_test_SI 
b = -k_test_SI * r_eq_SI
c = 0.5 * k_test_SI * (r_eq_SI ** 2) - left
zeros_n = (-b - np.sqrt((b ** 2) - 4 * a * c)) / (2 * a)
zeros_p = (-b + np.sqrt((b ** 2) - 4 * a * c)) / (2 * a)
x0_angstrom = zeros_n * 10 ** 10
x0_au = x0_angstrom / psi4.constants.bohr2angstroms
print(x0_au)

## Calculation of the Huang-Rhys Factor
Huang Rhys factor can be calculated by both

$$ S = 1/2(\Delta x / x_0)^2 \tag{Turner}$$

from the mode anharmonicity paper

and 

$$ S = \frac{m\omega_{vib} \Delta x^2}{2 \hbar} \tag{Hsu}$$

from the polaritonic Huang-Rhys factor paper

In [ ]:
# Constants and Variables
x_0 = wfn_H2.frequency_analysis["Xtp0"].data[5] 
k_psi = wfn_H2.frequency_analysis["k"].data[5]
delta_au = (r_data_e[min_S1_loc] - r_data[min_S0_loc]) / psi4.constants.bohr2angstroms
delta_m = (r_data_e[min_S1_loc] - r_data[min_S0_loc]) * 10 ** (-10)
delta_angstrom = (r_data_e[min_S1_loc] - r_data[min_S0_loc])
omega_vib = np.sqrt(k_test_SI / mu_kg)
h_bar = constants.hbar
r_eq_au = r_eq_ang / psi4.constants.bohr2angstroms
x0_test = np.sqrt(h_bar * omega_vib / k_test_SI)
x0_test_au = (x0_test * 10 ** (10))  / psi4.constants.bohr2angstroms
print(x0_test_au)

# Turner
S_Turner = 0.5 * (delta_au / x0_test_au) ** 2

# Hsu
S_Hsu = mu_kg * omega_vib * delta_m ** 2 / (2 * h_bar)

print(S_Turner)
print(S_Hsu)

First plot the ground-state potential energy surfaces for $ \text{H}_2 $ inside and outisde the cavity.  The effect of the cavity will raise the energy slightly.

In [ ]:
plt.plot(r_data, fci_S0, label='cavity free |g>')
plt.plot(r_data_e, fci_S1_e, label='cavity free |e>')
#plt.plot(r_array, cavity_free_E_array_LiH[:,1], 'b-', label='cavity free LiH |e>')
#plt.plot(r_array, V_fit, label='fit')
plt.xlim(0.6, 1.45)
plt.legend()
plt.show()